# This notebook is a combination of shared functions accross our voila notebooks

In [1]:
from ipywidgets import HBox, VBox, IntSlider, interactive_output, Dropdown, FloatSlider
from IPython.display import display
import numpy as np
import matplotlib.pyplot as plt
import math
from mpl_toolkits import mplot3d
import frispy
from dataclasses import dataclass
import json

In [2]:
if (frispy.__version__ < '1.1.0'):
    print('You are using frispy version '+frispy.__version__+'. To upgrade your requirements, run `pip install --upgrade --force-reinstall -r requirements.txt`. Also see readme if you have issues with Frispy.')
assert(frispy.__version__ >= '1.1.0')

In [ ]:
output_file_name = 'coefficients_optimizer_results4.txt'

In [3]:
# taken from stack overflow
def set_axes_equal(ax):
    '''Make axes of 3D plot have equal scale so that spheres appear as spheres,
    cubes as cubes, etc..  This is one possible solution to Matplotlib's
    ax.set_aspect('equal') and ax.axis('equal') not working for 3D.

    Input
      ax: a matplotlib axis, e.g., as output from plt.gca().
    '''

    x_limits = ax.get_xlim3d()
    y_limits = ax.get_ylim3d()
    z_limits = ax.get_zlim3d()

    x_range = abs(x_limits[1] - x_limits[0])
    x_middle = np.mean(x_limits)
    y_range = abs(y_limits[1] - y_limits[0])
    y_middle = np.mean(y_limits)
    z_range = abs(z_limits[1] - z_limits[0])
    z_middle = np.mean(z_limits)

    # The plot bounding box is a sphere in the sense of the infinity
    # norm, hence I call half the max range the plot radius.
    plot_radius = 0.5*max([x_range, y_range, z_range])

    ax.set_xlim3d([x_middle - plot_radius, x_middle + plot_radius])
    ax.set_ylim3d([y_middle - plot_radius, y_middle + plot_radius])
    ax.set_zlim3d([0, (z_middle + plot_radius)*2])

def get_ax(fig,x,y,z,phi,subplotnum, title,azim=0,elev=0):
    radius = 0.2
    X = x
    Y = y
    Z = z
    X1 = X
    X2 = X
    Y1 = Y + radius * np.cos(phi)
    Y2 = Y - radius * np.cos(phi)
    Z1 = Z + radius * np.sin(phi)
    Z2 = Z - radius * np.sin(phi)
    
    ax = fig.add_subplot(subplotnum, projection='3d')
    ax.set_proj_type('ortho')
    ax.plot3D(X, Y, Z, 'red')
    ax.plot3D(X1, Y1, Z1, 'red')
    ax.plot3D(X2, Y2, Z2, 'red')
    ax.view_init(elev, azim)
    set_axes_equal(ax)
    ax.set_box_aspect([1,1,1])
    ax.set_title(title)
    

def interactive_refresh_plot(x,y,z,phi):
    fig = plt.figure(figsize = [14, 14])
    
    ax1 = get_ax(fig,x,y,z,phi,221,'3d view', 135,25)
    ax2 = get_ax(fig,x,y,z,phi,222, 'side view', -90)
    ax3 = get_ax(fig,x,y,z,phi,223, 'top view', 180, 90)
    ax4 = get_ax(fig,x,y,z,phi,224, 'thrower\'s view', 180)
    
    plt.show()

In [4]:
def compute_custom_trajectory(
        PL0,
        PLa,
        PD0,
        PTy0,
        PTya,
        x,
        y,
        z,
        vx,
        vy,
        vz,
        phi,
        theta,
        gamma,
        dphi,
        dtheta,
        dgamma,
        alpha_0 = -4 * np.pi / 180, # THIS IS AN ISSUE WITH FRISPY. THIS SHOULD BE -4 not 4
        PDa = None,
        PTxwx = None,
        PTxwz = None,
        PTywy = None,
        PTzwz = None,
        area = None,  # m^2
        I_zz = None,  # kg*m^2
        I_xx = None,  # kg*m^2
        mass = None,  # kg
    ):
    disc = frispy.Disc()
    disc.model.PL0 = PL0
    disc.model.PLa = PLa
    disc.model.PD0 = PD0
    disc.model.PTy0 = PTy0
    disc.model.PTya = PTya
    disc.model.alpha_0 = alpha_0
    if PDa is not None:
        disc.model.PDa = PDa
    if PTxwx is not None:
        disc.model.PTxwx = PTxwx
    if PTxwz is not None:
        disc.model.PTxwz = PTxwz
    if PTywy is not None:
        disc.model.PTywy = PTywy
    if PTzwz is not None:
        disc.model.PTzwz = PTzwz
    if area is not None:
        disc._default_physical_attributes['area'] = area
    if I_zz is not None:
        disc._default_physical_attributes['I_zz'] = I_zz
    if I_xx is not None:
        disc._default_physical_attributes['I_xx'] = I_xx
    if mass is not None:
        disc._default_physical_attributes['mass'] = mass
    disc.default_initial_conditions['x'] = x
    disc.default_initial_conditions['y'] = y
    disc.default_initial_conditions['z'] = z
    disc.default_initial_conditions['vx'] = vx
    disc.default_initial_conditions['vy'] = vy
    disc.default_initial_conditions['vz'] = vz
    disc.default_initial_conditions['phi'] = phi
    disc.default_initial_conditions['theta'] = theta
    disc.default_initial_conditions['gamma'] = gamma
    disc.default_initial_conditions['dphi'] = dphi
    disc.default_initial_conditions['dtheta'] = dtheta
    disc.default_initial_conditions['dgamma'] = dgamma
    disc.reset_initial_conditions()
    result = disc.compute_trajectory(flight_time=10,n_times=1000)
    return result

In [5]:
def plot_func(
        PL0,
        PLa,
        PD0,
        PTy0,
        PTya,
        x,
        y,
        z,
        vx,
        vy,
        vz,
        phi,
        theta,
        gamma,
        dphi,
        dtheta,
        dgamma,
        alpha_0 = -4 * np.pi / 180, # THIS IS AN ISSUE WITH FRISPY. THIS SHOULD BE -4 not 4
        PDa = None,
        PTxwx = None,
        PTxwz = None,
        PTywy = None,
        PTzwz = None,
        area = None,  # m^2
        I_zz = None,  # kg*m^2
        I_xx = None,  # kg*m^2
        mass = None,  # kg
    ):
    result = compute_custom_trajectory(
        PL0,
        PLa,
        PD0,
        PTy0,
        PTya,
        x,
        y,
        z,
        vx,
        vy,
        vz,
        phi,
        theta,
        gamma,
        dphi,
        dtheta,
        dgamma,
        alpha_0 = alpha_0,
        PDa = PDa,
        PTxwx = PTxwx,
        PTxwz = PTxwz,
        PTywy = PTywy,
        PTzwz = PTzwz,
        area = area,
        I_zz = I_zz,
        I_xx = I_xx,
        mass = mass
    )
    times = result.times
    x, y, z, phi = result.x, result.y, result.z, result.phi
    interactive_refresh_plot(x,y,z, phi)

In [6]:
class Result:
    def __init__(self, v, x, y):
        self.values = v
        self.end_x = x
        self.end_y = y
    def toString(self):
        return json.dumps({
            'values': self.values,
            'x': self.end_x,
            'y': self.end_y
        });